In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
from torch import nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.inputSize = (8) #one-hot encoding of spin outcomes
        self.outputSize = 1
        self.hiddenSize = 2
        
        self.b1 = torch.randn(4, requires_grad=True) #4 biases corresponding to each outcome for qubit1 
        self.b2 = torch.randn(4, requires_grad=True) #4 4 biases corresponding to each outcome for qubit2 

    def forward(self, X):
        # define information flow within the network: input (two one-hot encoded vecs -> P)
        self.z = torch.tensor([torch.dot(self.b1, X[0:4]), torch.dot(self.b2, X[4:])], requires_grad=True) #compute a1b1, a2b2
                #self.z2 = nn.Softmax(self.z, dim=0)
        self.z2 = torch.exp(self.z)
        #self.z2 = torch.tensor([self.softmax(self, 0, X[:4]), self.softmax(self, 1, X[4:])]) #softmax the result
        self.z3 = torch.prod(self.z2) #calculate P = P(a_1) * P(a_2|a_1)
        
        return self.z3 #1d output: just a probability
    
    
    def backward(self, X, y, o):
        # backpropagation to update network parameters
        # to update: b1 biases, b2 biases
        
        self.o_error = y - o #scalar: error in output
        
        update_1 = self.softmax(self, 1, X[4:]) * self.softmax(self, 0, X[:4]) * (1 - self.softmax(self, 0, X[:4]))
        update_2 = self.softmax(self, 0, X[:4]) * self.softmax(self, 1, X[4:]) * (1 - self.softmax(self, 1, X[4:]))
        
        #print(update_1)
        #print(update_2)
        
        X_lis = X.tolist()
        X_1 = X_lis[:4]
        X_2 = X_lis[4:]
        
        
        self.b1[X_1.index(1)] += self.o_error * update_1
        self.b2[X_2.index(1)] += self.o_error * update_2
    
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        loss = y - o
        optim = torch.optim.SGD([self.b1, self.b2], lr=1e-1)
        loss.backward()
        #optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
        optim.step()
        print(self.b1)
        print(self.b2)
        #self.backward(X, y, o)
        
    def saveWeights(self, model):
        torch.save(model, "NN")
        
    def softmax(self, t, spin, arg):
        #print(self)
        #print(spin)
        #print(arg)
        if spin == 0:
            b = self.b1
        elif spin == 1:
            b = self.b2
        return(np.exp(torch.dot(arg,b)) / (np.exp(torch.dot(torch.tensor([1, 0, 0, 0],dtype=torch.float),b)) * np.exp(torch.dot(torch.tensor([0, 1, 0, 0],dtype=torch.float),b)) * np.exp(torch.dot(torch.tensor([0, 0, 1, 0],dtype=torch.float) , b)) * np.exp(torch.dot(torch.tensor([0, 0, 0, 1],dtype=torch.float) ,b))))
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))
        

In [3]:
NN = Network()

optimizer = torch.optim.SGD([NN.b1, NN.b2], lr=1e-1)
X = X_train
y = Y_train
for t in range(50):
    print('-' * 50) 
    optimizer.zero_grad()
    #print A.grad, b.grad
    y_pred = NN.forward(X[t])
    loss = ((y_pred - y[t]) ** 2).mean()
    print(t, loss.data)
    loss.backward()
    optimizer.step()

NameError: name 'X_train' is not defined

## Data Input

In [ ]:
%store -r P_init
%store -r measurements_test

def one_hot(val):
    out = [0, 0, 0, 0]
    out[val - 1] = 1
    
    return(out)

### Generate Training Data

In [81]:
from random import randrange

n_samples = 1000

X_train = torch.zeros(n_samples, 8)
Y_train = torch.zeros(n_samples)

for i in range(n_samples):
    r = randrange(16)
    meas = measurements_test[r]
    #print(torch.tensor(one_hot(meas[0]) + one_hot(meas[1])))
    X_train[i] = torch.tensor(torch.tensor(one_hot(meas[0]) + one_hot(meas[1])))
    Y_train[i] = P_init[r][0]
    
    


/Home/hh234/linux/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


### Train the network

In [82]:
NN = Network()
for i in range(n_samples):  # trains the NN 1,000 times
    X = X_train[i]
    Y = Y_train[i]
    print ("#" + str(i) + " Loss: " + str(torch.mean((Y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, Y)
    #print(NN(X))
NN.saveWeights(NN)

#0 Loss: 0.0036078928969800472
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#1 Loss: 0.1650179922580719
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#2 Loss: 0.016797848045825958
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#3 Loss: 0.03848676756024361
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#4 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#5 Loss: 2.383208751678467
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#6 Loss: 0.4810550808906555
tensor([ 0.6927,  0.

tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#106 Loss: 2.383208751678467
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#107 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#108 Loss: 3.212294340133667
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#109 Loss: 1.5190893411636353
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#110 Loss: 0.00038320577004924417
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#111 Loss: 0.32939061522483826
tensor([ 0.6927,  0.7858,  1.1386, -0.

#213 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#214 Loss: 0.0036078928969800472
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#215 Loss: 0.32939061522483826
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#216 Loss: 0.016797848045825958
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#217 Loss: 3.212294340133667
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#218 Loss: 4.981961727142334
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#219 Loss: 0.9974374771118164
tensor(

tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#321 Loss: 4.981961727142334
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#322 Loss: 0.0036078928969800472
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#323 Loss: 0.32939061522483826
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#324 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#325 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#326 Loss: 0.03582051023840904
tensor([ 0.6927,  0.7858,  1.1386, -0.

tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#425 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#426 Loss: 0.03848676756024361
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#427 Loss: 0.4810550808906555
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#428 Loss: 0.0036078928969800472
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#429 Loss: 0.00038320577004924417
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#430 Loss: 0.9974374771118164
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141

#529 Loss: 0.9974374771118164
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#530 Loss: 0.03582051023840904
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#531 Loss: 29.46666717529297
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#532 Loss: 29.46666717529297
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#533 Loss: 1.5190893411636353
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#534 Loss: 0.32939061522483826
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#535 Loss: 0.4810550808906555
tensor([ 0

tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#634 Loss: 0.00038320577004924417
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#635 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#636 Loss: 0.016797848045825958
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#637 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#638 Loss: 0.03582051023840904
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#639 Loss: 0.03582051023840904
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141,

tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#739 Loss: 29.46666717529297
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#740 Loss: 0.9974374771118164
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#741 Loss: 0.32939061522483826
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#742 Loss: 29.46666717529297
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#743 Loss: 0.9974374771118164
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#744 Loss: 0.00038320577004924417
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2

tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#843 Loss: 2.383208751678467
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#844 Loss: 0.4810550808906555
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#845 Loss: 0.03582051023840904
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#846 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#847 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#848 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349]

#947 Loss: 2.383208751678467
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#948 Loss: 0.03848676756024361
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#949 Loss: 0.03848676756024361
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#950 Loss: 73.62034606933594
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#951 Loss: 0.016797848045825958
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#952 Loss: 36.25737380981445
tensor([ 0.6927,  0.7858,  1.1386, -0.4152], requires_grad=True)
tensor([-0.2990, -1.1124,  1.0141, -2.6349], requires_grad=True)
#953 Loss: 2.383208751678467
tensor([ 0

In [62]:
P_init

array([[0.25      ],
       [0.08333333],
       [0.08333333],
       [0.08333333],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778]])

In [63]:
cum_P = np.cumsum(P_init)

In [64]:
NN.b1

tensor([ 0.5579, -0.5564,  0.3541,  0.4595], requires_grad=True)

In [65]:
NN.b2

tensor([ 0.1793, -0.8548, -1.4250,  0.4935], requires_grad=True)

Test if our model is useful at all

In [66]:
def calcProbs(a, b, C, outcome):
    print(outcome)
    P = 1/(C[0]*C[1]) * np.exp(a[outcome[0]-1]) * np.exp(b[outcome[1]-1])
    return P

def get_C(biases):
    C = 0
    for i in range(len(biases)):
        C += np.exp((i+1)*biases[i])
    return(C)

In [68]:
biases1 = NN.b1.detach().numpy()
biases2 = NN.b2.detach().numpy()
Cs = np.array([get_C(biases1), get_C(biases2)])

In [69]:
a = []
for outcome in measurements_test:
    a.append(calcProbs(biases1, biases2, Cs, outcome))

print(np.sum(np.array(a)))

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
0.1929215671511992


In [70]:
a

[0.021618916834398875,
 0.007686347531912426,
 0.004345684345071138,
 0.029599442762095765,
 0.007094159901425933,
 0.0025222437769201727,
 0.001426018697477232,
 0.009712937126123623,
 0.01763328795737328,
 0.006269304813412307,
 0.0035445209404090477,
 0.024142536908757003,
 0.019594022688475765,
 0.0069664206160488466,
 0.003938654203007644,
 0.026827068048290156]

In [253]:
P_init

array([[0.25      ],
       [0.08333333],
       [0.08333333],
       [0.08333333],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778],
       [0.08333333],
       [0.02777778],
       [0.02777778],
       [0.02777778]])

In [49]:

def gen_samples(cum_probs, measurements, n_samples):
    # takes a distribution of measurements and generates samples according 
    # to this distribution
    
    if len(cum_probs) == len(measurements):
        print("Values match")
    else: 
        print("Values don't match")
        
    samples = []
    
    cum_probs_prim = [0] + cum_probs
    
    for i in range(n_samples):
        n_rand = np.random.rand()
        for j in range(len(cum_probs_prim)-1):
            if (cum_probs_prim[j] <= n_rand) and (n_rand < cum_probs_prim[j+1]):
                samples.append(measurements[j])
    
    return(samples)
    

In [54]:
sample_set = gen_samples(cum_P, measurements_test, 100)

Values match


In [228]:
measurements

[(1,), (2,), (3,), (4,)]

In [55]:
sample_set

[(2, 4),
 (3, 1),
 (1, 2),
 (4, 3),
 (3, 4),
 (1, 2),
 (4, 2),
 (3, 2),
 (1, 2),
 (2, 1),
 (2, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (1, 2),
 (1, 3),
 (3, 4),
 (3, 3),
 (4, 1),
 (1, 1),
 (3, 4),
 (2, 4),
 (2, 4),
 (2, 4),
 (2, 4),
 (3, 2),
 (4, 3),
 (3, 3),
 (1, 2),
 (2, 4),
 (3, 3),
 (3, 4),
 (1, 1),
 (1, 4),
 (1, 3),
 (1, 3),
 (2, 3),
 (1, 2),
 (1, 4),
 (1, 4),
 (2, 4),
 (1, 3),
 (3, 4),
 (2, 1),
 (2, 1),
 (3, 1),
 (1, 3),
 (1, 3),
 (1, 4),
 (2, 2),
 (2, 2),
 (1, 4),
 (2, 2),
 (1, 4),
 (3, 2),
 (1, 1),
 (3, 1),
 (3, 4),
 (2, 1),
 (3, 4),
 (4, 3),
 (3, 4),
 (3, 4),
 (1, 2),
 (1, 1),
 (1, 3),
 (1, 1),
 (1, 1),
 (2, 4),
 (1, 1),
 (1, 1),
 (2, 1),
 (4, 3),
 (2, 2),
 (2, 4),
 (2, 4),
 (1, 4)]

In [65]:
a = torch.tensor([3, 2])

In [67]:
a[0] * 4

tensor(12)

In [69]:
c = torch.randn(2, 4)

In [70]:
c

tensor([[-0.5312,  1.1626,  0.2101, -0.3636],
        [-0.0905,  0.4233, -1.9423,  1.2532]])

In [71]:
c[0]

tensor([-0.5312,  1.1626,  0.2101, -0.3636])

In [72]:
torch.prod(torch.randn(2))

tensor(1.0005)

In [81]:
one_hot(4)

[0, 0, 0, 1]